In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import time
import matplotlib.pyplot as plt
%matplotlib inline

### Data Preparation

In [ ]:
dfn= pd.read_csv('archer-normal-network.csv', sep=';')
dfm= pd.read_csv('archer-attack-mirai-network.csv', sep=';', error_bad_lines=False)

In [ ]:
dfn = dfn.drop('Info', axis=1)
dfm = dfm.drop('Info', axis=1)

In [ ]:
dfn = dfn.drop('No.', axis=1)

In [ ]:
dfm = dfm.drop('No.', axis=1)

In [ ]:
dfem = pd.read_csv('archer-attack-mirai.csv', sep=';')
dfen = pd.read_csv('archer-normal.csv', sep=';')

In [ ]:
dfen = dfen.rename(columns={'time' : 'Energy_Time'})
dfem = dfem.rename(columns={'time' : 'Energy_Time'})

In [ ]:
dfem = dfem.drop('timestamp', axis=1)
dfen = dfen.drop('timestamp', axis=1)

In [ ]:
dfem.plot(x='Time', y='energy',  figsize=(10, 5), grid=True)

In [ ]:
dfen.plot(x='Time', y='energy',  figsize=(10, 5), grid=True)

In [ ]:
dfen.describe()

In [ ]:
dfm = dfm[dfm.Protocol != 'ICMP']

In [ ]:
dfn = dfn[dfn.Protocol != 'ICMP']

In [ ]:
for col in dfn:
    print(dfn['Protocol'].unique())

In [ ]:
dfn.groupby('Protocol').count()

In [ ]:
dfn['Protocol'].value_counts().plot.bar()

In [ ]:
dfm['TCP_Protocol'] = np.where(dfm['Protocol'] == 'TCP', 1, 0)
dfm

In [ ]:
dfn['TCP_Protocol'] = np.where(dfn['Protocol'] == 'TCP', 1, 0)
dfn

In [ ]:
dfen['target'] = 'Normal'
dfem['target'] = 'Mirai attack'

In [ ]:
dfen

In [ ]:
Normal_merge

In [ ]:
Normal_merge.corr()

In [ ]:
# Finding categorical features
num_cols = Normal_merge._get_numeric_data().columns

cate_cols = list(set(Normal_merge.columns)-set(num_cols))
cate_cols


In [ ]:
Final_merge = Normal_merge.append(Mirai_merge)
display(Final_merge)

In [ ]:
Final_merge.groupby('Protocol').count()

In [ ]:
Final_merge = Final_merge[Final_merge.Protocol != 0]

In [ ]:
for col in Final_merge:
    print(Final_merge['Protocol'].unique())

In [ ]:
# protocol_type feature mapping
pmap = {'DNS':0, 'ARP':1, 'DHCP':2, 'ICMP':3, 'TCP':4, 'TLSv1.2':5, 'NTP':6, 'ICMPv6':7}
Final_merge['Protocol'] = Final_merge['Protocol'].map(pmap)

In [ ]:
from sklearn import preprocessing


In [ ]:
le = preprocessing.LabelEncoder()
Final_merge['Source'] = le.fit_transform(Final_merge.Source.values)
Final_merge['Destination'] = le.fit_transform(Final_merge.Destination.values)

In [ ]:
Final_merge

In [ ]:
Final_merge

In [ ]:
Final_merge.to_csv('FinalSet.csv')

# The Model

In [ ]:
import numpy as np
from collections import Counter

In [ ]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

In [ ]:
class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [ ]:
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
                or n_labels == 1
                or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        
        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

In [ ]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]

In [ ]:
def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [ ]:
class RandomForest:
    
    def __init__(self, n_trees=10, min_samples_split=2,
                 max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_samples_split,
                max_depth=self.max_depth, n_feats=self.n_feats)
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

In [ ]:
# Target variable and train set
y = Final_merge[['target']]
X = Final_merge.drop(['target', ], axis = 1)
  
# Split test and train data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.6, random_state = 42)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
clf = RandomForest(n_trees = 5)
clf.fit(X_train, y_train)

### Results

In [ ]:
clf_cv_score = cross_val_score(clf, X, y, cv=10, scoring='roc_auc')

In [ ]:
print("=================== Confusion Matrix ======================")
print(confusion_matrix(y_test, y_pred))
print('\n') 
print("=================== Classification Report =================")
print(classification_report(y_test, y_pred))
print('\n') 
print("=================== All AUC Scores =======================")
print(clf_cv_score)
print('\n') 
print("=================== Mean AUC Score =======================")
print("Mean AUC Score - Random Forest: ", clf_cv_score.mean())

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
import time
from subprocess import check_output
import warnings


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# find best scored 5 features
select_feature = SelectKBest(chi2, k=5).fit(X_train, y_train)

In [ ]:
print('Score list:', select_feature.scores_)
print('Feature list:', X_train.columns)

In [ ]:
from sklearn.feature_selection import RFECV

# The "accuracy" scoring is proportional to the number of correct classifications
clf_rf_4 = RandomForestClassifier() 
rfecv = RFECV(estimator=clf_rf_4, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(X_train, y_train)

print('Optimal number of features :', rfecv.n_features_)
print('Best features :', X_train.columns[rfecv.support_])

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score of number of selected features")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
clf_rf_5 = RandomForest()      
clr_rf_5 = clf_rf_5.fit(X_train,y_train)
importances = clr_rf_5.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest

plt.figure(1, figsize=(10, 8))
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices],
       color="g", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), X_train.columns[indices],rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.show()
plt.savefig('books_read.png')